This notebook calculates the different pCO2 components needed to calculate the pCO2-Residual prior to algorithm processing. 

This notebook was originally created by Val Bennington, and modified by Thea Hatlen Heimdal.

In [ ]:
# Modules

import pandas as pd
import xarray as xr
import numpy as np
import numpy.ma as ma
%matplotlib inline
%config InlineBackend.figure_format = 'jpg'
%config InlineBackend.print_figure_kwargs = {'dpi':200, 'bbox_inches': 'tight'}
import matplotlib as mpl
from matplotlib.ticker import AutoMinorLocator
import matplotlib.pyplot as plt
import scipy
import sklearn.linear_model 
import pickle
import cmocean as cm

# "pCO2-DIC" = pCO2 - pCO2_T

In [2]:
# pCO2-T at observed temp T = pCO2 (annual mean) * exp(0.0423*(Tobs - Tannualmean))

# Takahashi 2002:
# Takahashi, T., et al. (2002), Global sea-air CO2 flux based on climatological surface ocean pCO2, and seasonal biological and temperature effects, Deep Sea Res., Part II, 49, 1601–1622, doi:10.1016/S0967-0645(02)00003-6.

# pCO2-nonT at observed temperature = pCO2 * exp(0.0423*(SST-SSTmean))

In [1]:
def pCO2_components(pco2_series,sst_series):
    
    # Use climatological pCO2 to calculate pCO2_T
    pCO2_T = pco2_series.mean("time")*np.exp(0.0423* (sst_series - sst_series.mean("time")))

    pCO2_T = pCO2_T.transpose("time","ylat","xlon")
    # pCO2_nonT = pCO2 - pCO2_T 
      
    pCO2_nonT = pco2_series * np.exp(0.0423 * (sst_series.mean("time") - sst_series))
    
    return pCO2_T, pCO2_nonT
    

In [2]:
def pCO2_from_pCO2_nonT(pco2_nonT,sst_series):
    
    pco2_series = pco2_nonT / (np.exp(0.0423 * (sst_series.mean("time") - sst_series)))
    
    return pco2_series

In [6]:
member_dir = "/local/data/artemis/workspace/jfs2167/recon_eval"
reference_output_dir = f"{member_dir}/references"
ensemble_dir_head = "/data/artemis/simulations/LET"
run_dir = "/data/artemis/workspace/vbennington/sampling/saildrone_5yr/models/reconstructions/xg"

# Loading references
path_LET = f"{reference_output_dir}/members_LET_dict.pickle"

with open(path_LET,'rb') as handle:
    mems_dict = pickle.load(handle)

# Calculate pCO2 components for LET

In [ ]:
#Loop through each model and ensemble member:

for ens, mem_list in mems_dict.items():
    print(ens)
    for member in mem_list:
        print(member)
        
        member_dir = f"{ensemble_dir_head}/{ens}/member_{member}"
        
        
        if ens == "CanESM2":
            # CanESM2 files are mislabeled as going to 201712
            sst_path = f"{member_dir}/SST_2D_mon_{ens}{member}_1x1_198201-201712.nc"
            pco2_path = f"{member_dir}/pCO2_2D_mon_{ens}{member}_1x1_198201-201712.nc"
            file_out = f"/data/artemis/workspace/vbennington/LET/{ens}/pCO2_components_{ens}{member}_198201-201712.nc"
        else:
            sst_path = f"{member_dir}/SST_2D_mon_{ens}{member}_1x1_198201-201701.nc"
            pco2_path = f"{member_dir}/pCO2_2D_mon_{ens}{member}_1x1_198201-201701.nc"
            file_out = f"/data/artemis/workspace/vbennington/LET/{ens}/pCO2_components_{ens}{member}_198201-201701.nc"

        # Load modeled SST and pCO2:
        sst_series = xr.open_dataset(sst_path).SST
        pco2_series = xr.open_dataset(pco2_path).pCO2   
        
        sst_series = sst_series.assign_coords({"time": pco2_series.time})
        
        
        pco2_T, pco2_nonT = pCO2_components(pco2_series,sst_series)
        
        pCO2_pCO2T_diff = pco2_series - pco2_T   # Difference between pCO2 (total) and pCO2-T
        
        
        comp = xr.Dataset({
                        'pCO2':(["time","ylat","xlon"],pco2_series),
                        'pCO2_T':(["time","ylat","xlon"],pco2_T),
                        'pCO2_nonT':(["time","ylat","xlon"],pco2_nonT),
                        'pCO2_pCO2T_diff':(["time","ylat","xlon"],pCO2_pCO2T_diff)},
                        coords={'time': (['time'],sst_series.time.values),
                        'ylat': (['ylat'],sst_series.ylat.values),
                        'xlon':(['xlon'],sst_series.xlon.values)})

        comp.to_netcdf(f"{file_out}")
        
        del sst_path, pco2_path, sst_series, pco2_series, pco2_T, pco2_nonT, pCO2_pCO2T_diff